In [6]:
import IPython.display as ipd
s = open('texAbbr.tex','r').read()
print s
ipd.display_latex(s)

s = '$$%s\n$$'%s
print s
ipd.display_markdown(s)




\newcommand{\KLDiv}{\mathbb{KL}}
\newcommand{\bE}{\mathbb{E}}
\newcommand{\bH}{\mathbb{H}}
\newcommand{\const}{\text{const}}
$$
\newcommand{\KLDiv}{\mathbb{KL}}
\newcommand{\bE}{\mathbb{E}}
\newcommand{\bH}{\mathbb{H}}
\newcommand{\const}{\text{const}}
$$


In [2]:
%%bash
date

Sun 17 Jun 11:57:26 BST 2018


## 21.5 变分贝叶斯（变贝）

目前为止我们主要专注于在模型参数 $\theta$ 已知的情况下推断隐变量 $z_i$ 的分布。现在起我们开始考虑推断模型参数其本身。如果我们作一个完全的因子分解（也就是平均场）近似 $p(\theta | D) \approx \prod_k q(\theta_k)$，我们实际上就在做**变分贝叶斯**(VB:Variational Bayes, @Hinton&Camp1993,@McKay1995a,@Attias2000,@Beal&Ghahramani2006,@Smidl&Quinn2005)了。接下来我们会给出一些在假设不存在隐变量情况下，应用变分贝叶斯的例子，如果我们想要同时推断隐变量和参变量，并作如下近似 $p(\theta,z_{1:N}|D) \approx q(\theta) \prod_i q_i(z_i)$，我们实际上就在使用变分贝叶斯期望最大化算法(VBEM)，详见Sec21.6\ref{sec:21.6}。

### 21.5.1 实例：单一高斯变量的变贝

根据(@MacKay2003,p429)，我们考虑推断一维高斯分布的参数分布 $p(\mu,\lambda | D)$,此处$\lambda=1/\sigma^2$ 指高斯精度。为简便期间，我们使用一个如下的共轭先验分布

$$
p(\mu,\lambda) = \mathcal{N}(\mu|\mu_0,(\kappa \lambda)^{-1})\text{Ga}(\lambda|a_0,b_0)
~~~~\text{(21.65)}\label{eqn:21.65}
$$

但是呢，我们用来近似后验的是一个如下的因子分布

$$
q(\mu,\lambda) = q_\mu(\mu) q_\lambda(\lambda)
~~~~\text{(21.66)}\label{eqn:21.66}
$$

我们不需要指定分布 $q_\mu,\,q_\lambda$ 的确切形式，因为其最优形式会在推导的时候自动“出现”(方便的是，它们恰巧是相应的高斯分布和伽马分布)

由于我们在Sec4.6.3.7\ref{sec:4.6.3.7}已经得出了计算该模型确切后验的方法，因此如上处理的动机并不是很显然。动机主要有如下两条：首先这是一个有教学意义的练习，我们可以乘机通过与确切分布的对比对比，来分析近似分布的质量；其次，这个方法可以很自然地加以改造以适用一个半共轭的先验 $p(\mu,\lambda)=\mathcal{N}(\mu|\mu_0,\tau_0)\text{Ga}(\lambda|a_0,b_0)$，而这种先验下是无法进行确切推断的。

#### 21.5.1.1 目标分布

未正规化的对数后验具有形式

$$
\begin{aligned}
\log \tilde{p}(\mu,\lambda) = \log p(\mu,\lambda,D) 
&= \log p(D|\mu,\lambda) + \log p(\mu|\lambda) + \log p(\lambda)
~~~~\text{(21.67)}\label{eqn:21.67}
\\
&=\frac{N}{2}\log\lambda - {\lambda \over 2 } \sum_{i=1}^N (x_i - \mu)^2 - \frac{\kappa_0 \lambda}{2}(\mu-\mu_0)^2 
\\ & ~~ +{1\over 2}\log(\kappa_0 \lambda) + (a_0 - 1)\log \lambda - b_0 \lambda + \text{const}  
~~~~\text{(21.68)}\label{eqn:21.68}
\end{aligned}
$$

#### 21.5.1.2 更新 $q_\mu（\mu）$

通过对 $\lambda$ 求期望可以得到 $q_\mu(\mu)$ 的最优形式：

$$
\begin{aligned}
\log q_\mu(\mu) &= \bE_{q_\lambda}[\log p(D|\mu,\lambda) + \log p(\mu|\lambda) ] + \text{const} 
~~~~\text{(21.69)}\label{eqn:21.69}
\\
&= -\frac{\bE[\lambda]}{2} \left\{
\kappa_0 (\mu - \mu_0)^2 + \sum_{i=1}^N (x_i - \mu)^2
\right \} + \text{const}
~~~~\text{(21.70)}\label{eqn:21.70}
\end{aligned}
$$

通过展开平方项（？疑）可以证明 $q_\mu(\mu) = \mathcal(\mu | \mu_N , \kappa^{-1}_N)$,且有

$$
\mu_N={ \kappa_0 \mu_0 + N \bar{x} \over \kappa_0 + N }, \kappa_N = (\kappa_0 + N ) \bE_{q_\lambda}[\lambda]
~~~~\text{(21.71)}\label{eqn:21.71}
$$

此时我们还不知道 $q_\lambda(\lambda)$ 的确切分布所以无法计算 $\bE[\lambda]$，但我们马上就会解决这个玩意

#### 21.5.1.3 更新 $q_\lambda (\lambda)$

$q_\lambda(\lambda)$ 的最优分布有如下表示

$$
\begin{aligned}
\log q_\lambda(\lambda) 
&= \bE_{q_\mu}[ 
\log p(D|\mu,\lambda) + \log p(\mu | \lambda ) + \log p(\lambda)  ] + \const
~~~~\text{(21.72)}\label{eqn:21.72}
\\
&= (a_0 - 1 )\log \lambda -　b_0 \lambda + {1\over2}\log\lambda + {N\over2} \log\lambda 
\\
&~~-{\lambda\over2}\bE_{q_\mu}\left[
\kappa_0 (\mu - \mu_0)^2 +  \sum_{i=1}^N (x_i-\mu)^2
\right] + \const
~~~~\text{(21.73)}\label{eqn:21.73}
\end{aligned}
$$

可以发现这对应一个伽马分布，因而有　$q_\lambda(\lambda)=Ga(\lambda|a_N,b_N)$,且有

$$
\begin{aligned}
a_N &= a_0 + \frac{N+1}{2}
~~~~\text{(21.74)}\label{eqn:21.74}
\\
b_N &= b_0 + \frac{1}{2}\bE_{q_\mu}\left[
\kappa_0 (\mu - \mu_0)^2 +  \sum_{i=1}^N (x_i-\mu)^2
\right]
~~~~\text{(21.75)}\label{eqn:21.75}
\end{aligned}
$$

In [4]:
!date

Sun 17 Jun 13:40:46 BST 2018


#### 21.5.1.4 计算期望

为了实行上述更新，我们必须确定如何计算诸多期望值。鉴于 $q(\mu) = \mathcal(\mu | \mu_N,\kappa_N^{-1})$,我们得出

$$
\begin{aligned}
\bE_{q_(\mu)}[\mu] &= \mu_N
~~~~\text{(21.76)}\label{eqn:21.76}
\\
\bE_{q(\mu)}[\mu^2] &= {1\over \kappa_N} + \mu_N^2
~~~~\text{(21.77)}\label{eqn:21.77}
\end{aligned}
$$

鉴于 $q(\lambda) = \text{Ga}(\lambda \gvn a_N, b_N)$，我们有

$$
\bE_{q(\lambda)}[\lambda] = \frac{a_N}{b_N}
~~~~\text{(21.78)}\label{eqn:21.78}
$$

如此我们可以显式地给出更新方程。对 $q(\mu)$ 有

$$
\begin{aligned}
\mu_N 
&= \frac{\kappa_0 \mu_0 + N \bar{x}}{\kappa_0 + N}
~~~~\text{(21.79)}\label{eqn:21.79}
\\
\kappa_N 
&= (\kappa_0 + N ) \frac{a_N}{b_N}
~~~~\text{(21.80)}\label{eqn:21.80}
\end{aligned}
$$

相应地对 $q(\lambda)$ 有

$$
\begin{aligned}
a_N &= a_0 + \frac{N+1}{2}
~~~~\text{(21.81)}\label{eqn:21.81}
\\
b_N &= b_0 + \kappa_0 (\bE[\mu^2] +\mu_0^2 -2\bE[\mu]\mu_0 ) 
+ \frac{1}{2}\sum_{i=1}^N (x_i^2 + \bE[\mu^2] -2 \bE[\mu ] x_i)
~~~~\text{(21.82)}\label{eqn:21.82}
\end{aligned}
$$

可以看到 $\mu_N$ 和 $a_N$ 实际上是固定的常数，且只有 $\kappa_N\,,b_N$需要被迭代更新。(事实上，我们可以利用迭代方程，解析地求出$\kappa_N\,\,b_N$的不动点，但是在此不作展示，只介绍迭代更新法)


#### 21.5.1.5 展示

配图Fig21.5\ref{fig:21.5}给出了一个工作示意。绿色的轮廓表示确切的高斯-伽马后验分布，虚线的轮廓表示的是变分近似在不同的迭代步的结果。可以看到最终的近似是和确切解较为相似的。然而，近似解比真实分布更为“紧致”。事实上平均场近似推断常常欠估计后验的不确定性，更多有关讨论请见Sec21.2.2\ref{sec:21.2.2}。

#### 21.5.1.6 下界

在变贝中，我们最大化的是 $L(q)$ 是数据的边际似然值的下界。

$$
L(q) \le \log p(D) = \log \int \int p(D\mu) p(\mu, \lambda) d\mu d\lambda
~~~~\text{(21.83)}\label{eqn:21.83}
$$

出于多种原因，计算这个下界本身是很有用的。首先，下界可以用来检查算法是否收敛了；其次，它可以用来检查算法的正确性：和期望最大算法(EM)的情况一样，如果下界不是单调增加的话，那一定是哪里出幺蛾子了；再次，下界可以用作一个对数据边际似然的近似，因而可以用来做贝叶斯模型选择。

不幸的是，计算这个下界需要经过一些很是心累的代数处理。以下我们会给出这个特例的下界的细节，但我们将不经证明，甚至不加讨论地直接给出下界的结果，因为这样子会简洁许多（?我的天）。

对这个模型，我们可以如下计算似然函数 $L(q)$ 

$$
\begin{aligned}
L(q) 
&= \int \int q(\mu, \lambda) \log 
\frac{ p(D,\mu,\lambda) }
{q(\mu,\lambda) }
d\mu d\lambda
~~~~\text{(21.84)}\label{eqn:21.84}
\\
&= \bE [\log p(D\ gvn \mu,\lambda) ] 
+ \bE[ \log p(\mu \gvn \lambda) ] 
+ \bE[ \log p(\lambda) ] 
\\ 
& ~~ - \bE[\log q(u)] - \bE[ \log q(\lambda) ]  
~~~~\text{(21.85)}\label{eqn:21.85}
\end{aligned}
$$

以上所有的期望都是关于 $q(\mu,\lambda)$。可以看出最后两项不过是高斯和伽马分布的熵，因此可以直接写出

$$
\begin{aligned}
\bH ( \mathcal{N}(\mu_N,\kappa_N^{-1} ) &= 
-\frac{1}{2} \log \kappa_N + \frac{1}{2} (1 + \log (2\pi))
~~~~\text{(21.86)}\label{eqn:21.86}
\\
\bH(\text{Ga(a_N,b_N)}) 
&= \log \Gamma(a_N) - (a_N - 1) \psi (a_N) - \log (b_N) + a_N
~~~~\text{(21.87)}\label{eqn:21.87}
\end{aligned}
$$

此处 $\psi()$ 是双伽马函数。

为了计算其它的项目，我们要用到如下结果

$$
\begin{aligned}
\bE [ \log x \gvn x \sim \text{Ga}(a,b)] 
&= \psi(a) - \log(b) 
~~~~\text{(21.88)}\label{eqn:21.88}
\\
\bE [ x \gvn x \sim \text{Ga}(a,b) ] 
&= {a \over b}
~~~~\text{(21.89)}\label{eqn:21.89}
\\
\bE [ x \gvn x \sim \mathcal{N}(\mu, \sigma^2) ]
&= \mu
~~~~\text{(21.90)}\label{eqn:21.90}
\\
\bE [ x^2 \gvn x \sim \mathcal{N} (\mu, \sigma^2)] 
&= \mu^2 + \sigma^2 
~~~~\text{(21.91)}\label{eqn:21.91}
\end{aligned}
$$


可以证明对数似然的期望符合下式

$$
\begin{aligned}
\bE_{q(\mu,\lambda)}&[ \log p(D \gvn \mu \lambda) ] 
~~~~\text{(21.92)}\label{eqn:21.92}
\\
&= -{N\over 2 }\log(2\pi) + {N\over 2} \bE_{q_\lambda}[\log \lambda] - \frac{\bE_{q_\lambda}[ \lambda]}{2} \sum_{i=1}^N 
\bE_{q(\mu)} [ (x_i - \mu)^2 ] 
~~~~\text{(21.93)}\label{eqn:21.93}
\\
&= -{N\over 2 }\log(2\pi) + {N\over 2} (\psi(a_N) - \log b_N)
\\
& ~~ - {N a_N \over 2 b_N} \left( \hat{\sigma}^2 + \bar{x}^2 - 
2\mu_N \bar{x} + \mu^2_N + \frac{1}{\kappa_N}
\right)
~~~~\text{(21.94)}\label{eqn:21.94}
\end{aligned}
$$

此处的 $\bar{x},\,\hat{\sigma}^2$ 对应观测平均和方差。

对$\lambda$ 的对数先验的期望可做如下处理

$$
\begin{aligned}
\bE_{q(\lambda)}[ \log p(\lambda) ] 
~~&=~~ (a_0 - 1)\bE[ \log \lambda] -b_0 \bE [\lambda ] + a_0\log b_0 -\log \Gamma ( a_0 )
~~~~\text{(21.95)}\label{eqn:21.95}
\\
~~&=~~ (a_0 - 1)(\psi(a_N) - \log b_N) -b_0 \frac{a_N}{b_N}
+ a_0\log b_0 -\log \Gamma ( a_0 )
~~~~\text{(21.96)}\label{eqn:21.96}
\end{aligned}
$$

对$\mu$ 的对数先验的期望可做如下处理

$$
\begin{aligned}
\bE_{q(\mu, \lambda)}[ \log p(\mu \gvn \lambda) ] 
~~&=~~ \frac{1}{2} \log \frac{\kappa_0}{2\pi} + \frac{1}{2} 
\bE[ \log \lambda]q(\lambda) - \frac{1}{2} \bE_{q(\mu,\lambda)}
[ (\mu - \mu_0)^2 \kappa_0 \lambda ]
\\
~~&=~~ \frac{1}{2} \log \frac{\kappa_0}{2\pi} + 
\frac{1}{2}  (\psi(a_N) - \log b_N) 
\\
& ~~ - \frac{\kappa_0}{2} \frac{a_N}{b_N} 
\left[ (\mu_N - \mu_0)^2 + \frac{1}{\kappa_N}  \right]
~~~~\text{(21.97)}\label{eqn:21.97}
\end{aligned}
$$

将以上结果结合，可以得到对数似然的下界

$$
L(q) = {1 \over 2 }\log {1 \over \kappa_N} + \log \Gamma(a_N) - a_N \log b_N + \const
~~~~\text{(21.98)}\label{eqn:21.98}
$$

这个量随着变贝的迭代更新是单调递增的。

In [7]:
%%bash
date

Sun 17 Jun 15:08:31 BST 2018


### 21.5.2 实例： 线性回归中的变贝

***(？略略略）***

In [16]:
%%bash
ALI="21_Variational-Inference"
for FMT in markdown html
do
    jupyter nbconvert --TemplateExporter.exclude_code_cell=True ${ALI}*.ipynb --to $FMT
done
cat ${ALI}_*.md > .${ALI}.md


[NbConvertApp] Converting notebook 21_Variational-Inference_01-02.ipynb to markdown
[NbConvertApp] Writing 6542 bytes to 21_Variational-Inference_01-02.md
[NbConvertApp] Converting notebook 21_Variational-Inference_05.ipynb to markdown
[NbConvertApp] Writing 7713 bytes to 21_Variational-Inference_05.md
[NbConvertApp] Converting notebook 21_Variational-Inference_06.ipynb to markdown
[NbConvertApp] Writing 9441 bytes to 21_Variational-Inference_06.md
[NbConvertApp] Converting notebook 21_Variational-Inference_01-02.ipynb to html
[NbConvertApp] Writing 259214 bytes to 21_Variational-Inference_01-02.html
[NbConvertApp] Converting notebook 21_Variational-Inference_05.ipynb to html
[NbConvertApp] Writing 260156 bytes to 21_Variational-Inference_05.html
[NbConvertApp] Converting notebook 21_Variational-Inference_06.ipynb to html
[NbConvertApp] Writing 263444 bytes to 21_Variational-Inference_06.html


In [17]:
!ls -lhtr

total 12M
-rw-rw-r-- 1 shouldsee shouldsee  65K Jun 16 21:24 00 PreFace.pdf
-rw-rw-r-- 1 shouldsee shouldsee 2.6K Jun 16 21:24 00 PreFace.md
-rw-rw-r-- 1 shouldsee shouldsee  35K Jun 16 21:24 01 Introduction.md
-rw-rw-r-- 1 shouldsee shouldsee 698K Jun 16 21:24 01 Introduction.pdf
-rw-rw-r-- 1 shouldsee shouldsee  59K Jun 16 21:24 02 Probability.md
-rw-rw-r-- 1 shouldsee shouldsee  48K Jun 16 21:24 03 Generative models for discrete data.md
-rw-rw-r-- 1 shouldsee shouldsee 3.2M Jun 16 21:24 02 Probability.pdf
-rw-rw-r-- 1 shouldsee shouldsee  84K Jun 16 21:24 04 Gaussian models.md
-rw-rw-r-- 1 shouldsee shouldsee 2.2M Jun 16 21:24 03 Generative models for discrete data.pdf
-rw-rw-r-- 1 shouldsee shouldsee  80K Jun 16 21:24 05 Bayesian statistics.md
-rw-rw-r-- 1 shouldsee shouldsee 532K Jun 16 21:24 04 Gaussian models.pdf
-rw-rw-r-- 1 shouldsee shouldsee 3.6M Jun 16 21:24 05 Bayesian statistics.pdf
-rw-rw-r-- 1 shouldsee shouldsee 2.0K Jun 16 21:24 Abbreviations.md
-rw-rw-r-- 1 shouldsee